In [1]:
#import libraries
import numpy as np
from arqtic.program import Program, random_bitstring
from arqtic.hamiltonians import Ising_Hamiltonian
from arqtic.arqtic_for_ibm import run_ibm
import arqtic.QITE as qite
import qiskit as qk
from qiskit import Aer, IBMQ, execute

In [2]:
#define post-processing functions
#get amouot of work from QC results
def get_work(results, shots):
    work = 0
    for result in results:
        state_vec = result[0]
        count = result[1]
        work_comp = 0
        for i in range(len(state_vec)):
            if (state_vec[i] == 0):
                work_comp += 1
            else:
                work_comp += -1
        work_comp *= count
        work += work_comp
    work = work/shots
    return work

In [6]:
#define system variables
N = 2 #number of qubits
Jz = 0.1 #ising interaction strength
mu_x = 0.3 #transverse magnetic field strength
param_free_ham = Ising_Hamiltonian(2, Jz, [mu_x, 0, 0]) #parameter-free Hamiltonian
ising_ham0 = Ising_Hamiltonian(2, Jz, [0.01, 0, 0]) #Hamiltonian at beginning of parameter sweep

#define simulation variables
tau = 10 #total trajectory time to evolve lambda from 0 to 1
dtau = 1.0 #time-step for trajectory
num_steps = int(tau/dtau)
T = 1 #total number of trajectories
dt =  dtau #timestep for Trotter approximation: setting equal to dtau means one trotter-step per time-step in evolution
lamba_protocol = np.linspace((dtau/tau),1.0,num_steps)
dldt = 0.1 # d(lambda)/d(tau)
shots = 1000

#define QITE variables
beta = 2.0 #inverse temperature of systems
dbeta = 0.2 #step-size in beta for QITE
domain = 2 #domain of opertators for QITE

In [7]:
#interfacing with IBM
#load account
#qk.IBMQ.load_account()
#see available backends
#provider = qk.IBMQ.get_provider(group='open')
#print(provider.backends)
#set up simulator
simulator = Aer.get_backend('qasm_simulator')

In [8]:
#create program to move to x-basis for measurement
prog_xBasis = Program(N)
prog_xBasis.get_x_basis_prog()

#first state should be random
measured_metts_state = random_bitstring(N)
#subsequent entries are derived from running QMETTS on the previously derived
#state and measuring a random observable to get the state for the subsequent run

#need to sum work over each trajectory and then average over works
work = []
#loop over trajectories
for i in range(T):
    print(i)
    psi0 = qite.get_state_from_string(measured_metts_state)
    prog_TS = Program(N)
    prog_TS.make_ps_prog(measured_metts_state)
    print(measured_metts_state)
    print("PS circuit")
    prog_TS.print_list()
    prog_qite = Program(N)
    #note QITE algorithm should evolve state by beta/2 for temperature beta
    prog_qite.make_QITE_prog(ising_ham0, beta/2.0, dbeta, domain, np.asarray(psi0), 1.0)
    print("QITE circuit")
    prog_qite.print_list()
    prog_TS.append_program(prog_qite)
    print("TS circuit")
    prog_TS.print_list()
    #make and run qmetts program
    prog_qmetts = Program(N)
    prog_qmetts.append_program(prog_TS)
    #make random measurement operator
    if (i%2 == 0):
        prog_qmetts.append_program(prog_xBasis)
    #print("QMETTS circuit")
    #prog_qmetts.print_list()
    results = run_ibm(simulator, prog_qmetts,1)
    #update measured metts state for next trajectory
    measured_metts_state = results[0][0]
    #make and run JE program
    prog_JE = Program(N)
    prog_JE.append_program(prog_TS)
    #loop over time-steps in trajectory i
    work_i = 0    
    for step in range(num_steps):
        #print(step)
        #make Hamilton Evolution program for given time-step of given trajectory
        prog_hamEvol = Program(N)
        prog_hamEvol.make_hamEvol_prog(step, dtau, dt, lamba_protocol, param_free_ham)
        #complete JE program: combing IPS preparation, QITE, and Hamiltonian evolution
        prog_JE.append_program(prog_hamEvol)
        prog_JE.append_program(prog_xBasis)
        results = run_ibm(simulator, prog_JE, shots)
        #print(results)
        #print(get_work(results, shots))
        work_i += dldt*dtau*get_work(results, shots)
        work_i *= mu_x
    work.append(work_i)


print(work)
#avg_work = get_average work(work)
#FE = get_free energy(avg_work)

0
[0, 1]
PS circuit
X [1]
Aops is:  [[[0, 1], array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.08078267e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])], [[0, 1], array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.91425477e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])], [[0, 1], array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.58799043e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])], [[0, 1], array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e